In [1]:
import sys
sys.path.append("..")

import torch
from torch import nn
import torchmetrics
import numpy as np
import src.engine as engine
import src.experiments_maker as experiments_maker
import wandb
import time
import utils.config_manager as cm


/Users/nprianikov/Second-order-optimization/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
config = cm.create_config(epochs=1, lr=3e-3, batch_size=128, dataset='cifar10', optimizer='K_BFGS(L)', model='SmallCNN', wandb_log_batch=1)
with wandb.init(project="baselines_cnn", config=config, mode=cm.wandb_modes[0]):
    model, train_dataloader, test_dataloader, optimizer, criterion = experiments_maker.make(config, cm.device)
    engine.train(model, train_dataloader, test_dataloader, cm.loss_fn, optimizer, criterion, cm.device, config)

In [2]:
config = cm.create_config(epochs=1, batch_size=128, optimizer='SGD', model='DepthCNN', wandb_log_batch=32)
model, train_dataloader, test_dataloader, optimizer, criterion = experiments_maker.make(config, cm.device)